In [38]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_pinecone import Pinecone


In [39]:
# Load environment variables from .env file
load_dotenv()


True

In [40]:

def initialize_components():
    # Retrieve API keys from environment variables
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    openai_api_key = os.getenv("OPENAI_API_KEY")

    # Verify that API keys are loaded
    if not pinecone_api_key or not openai_api_key:
        raise ValueError("API keys for Pinecone or OpenAI are missing.")

    # Initialize LangChain components
    embeddings = OpenAIEmbeddings()
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.5)

    # Initialize Pinecone and set up vector store
    vectorstore = Pinecone(index_name="chatbot", embedding=embeddings)
    retriever = vectorstore.as_retriever()

    # Define the prompt template
    template = '''
    As a Medical AI assistant, you provide accurate and concise answers based on the given context.
    Follow these guidelines: If the information is not available within the context, state that you don’t know.
    Answer to the best of your ability, citing the provided documents as sources.
    Only use examples if explicitly requested, and avoid introducing examples outside the context.
    Ensure responses are limited to three or four sentences for clarity.
    Example:
    Question: "What is AIDS, and how does it relate to HIV?"
    Answer: "AIDS stands for acquired immunodeficiency syndrome, which is a condition caused by the human immunodeficiency virus (HIV). 
    HIV attacks and destroys specific white blood cells essential for the immune system. 
    While people infected with HIV may not show symptoms or have only minor illnesses for years, they are not considered to have AIDS until the virus causes severe damage to the immune system."
    Context:{context}
    {history}
    Question: {question}
    Answer
    '''

    prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template
    )

    # Initialize memory and retrieval chain
    memory = ConversationBufferWindowMemory(k=3, memory_key="history", input_key="question")

    retrieval_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt, "memory": memory}
    )

    return retrieval_chain, memory

In [41]:
def perform_query(retrieval_chain, query):
    response = retrieval_chain.invoke(query)
    print(f"Query: {query}\nResponse: {response}\n")


In [42]:
def main():
    # Initialize components
    retrieval_chain, memory = initialize_components()

    # Example queries
    perform_query(retrieval_chain, "What AIDS?")
    perform_query(retrieval_chain, "What is the capital of France?")
    perform_query(retrieval_chain, "What are different types of cancer?")
    perform_query(retrieval_chain, "Mammography?")
    perform_query(retrieval_chain, "Psychological impact of breast cancer diagnosis?")

    # Debugging memory state
    print("Memory state:", memory.load_memory_variables({}))

In [43]:
if __name__ == "__main__":
    main()

Query: What AIDS?
Response: {'query': 'What AIDS?', 'result': "AIDS stands for acquired immunodeficiency syndrome, a pattern of devastating infections caused by the human immunodeficiency virus (HIV), which attacks and destroys certain white blood cells that are essential to the body's immune system. Most people infected with HIV can live for years with no symptoms or only minor illnesses, but they do not have AIDS until the virus causes severe damage to the immune system. AIDS is a fatal disease, although treatments can extend life, and prevention of transmission remains the only method of control."}

Query: What is the capital of France?
Response: {'query': 'What is the capital of France?', 'result': 'I don’t know. The provided context does not contain information about the capital of France.'}

Query: What are different types of cancer?
Response: {'query': 'What are different types of cancer?', 'result': 'The context lists several types of cancer, including:\n\n1. **Leukaemia** - Ca